### 책소개 / 핵심문장 을 이용한 '생성문장', '해시태그' 생성 프롬프트 및 랭체인

In [34]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import json
from langchain_upstage import ChatUpstage
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
import os

# to generate sentence
sentence_system_template = SystemMessagePromptTemplate.from_template("""
당신은 작가의 문체를 학습하고 독자에게 전달하고자 하는 메시지를 생성하는 전문가입니다.
주어진 책의 소개글과 핵심 문장을 기반으로 작가의 문체를 반영한 짧은 메시지를 작성하세요.
메시지는 40자 이내이면서 두 문장으로 작성하되, 작가의 문체가 드러나야 합니다.
""")
sentence_user_template = HumanMessagePromptTemplate.from_template("""
아래 [text_description]는 이 책의 소개글 입니다. 제 3자의 시선으로 이 책에 대해 알려주는 글 입니다.
{text_description}

아래 [text_key_sentences]로부터 작가의 문체를 학습하세요. 작가의 핵심문장입니다.
{text_key_sentences}

위의 내용을 기반으로, 독자에게 작가가 전달하고 싶은 메시지를 40자 이내로 작성해주세요.
단, 이 작가가 쓴 글임을 알 수 있게 위의 핵심 문장의 문체를 반영한 문장으로 만들어 주세요.
""")
sentence_prompt_template = ChatPromptTemplate.from_messages([sentence_system_template, sentence_user_template])



# to generate hashtag
hashtag_system_template = SystemMessagePromptTemplate.from_template("""
당신은 책에 대한 정보를 바탕으로 적절한 해시태그를 생성하는 전문가입니다.
책의 소개글과 핵심 문장들을 분석하여 관련된 해시태그를 10개 작성하세요.
""")

hashtag_user_template = HumanMessagePromptTemplate.from_template("""
아래 [text_description]는 이 책의 소개글 입니다. 제 3자의 시선으로 이 책에 대해 알려주는 글 입니다.
{text_description}

아래 [text_key_sentences]는 이 책의 본문에서 추출한 핵심문장들입니다. 작가가 직접 쓴 문장이므로 문체가 담겨있습니다.
{text_key_sentences}

위 내용을 바탕으로 관련된 해시태그 10개를 뽑아주세요.
""")

hashtag_prompt_template = ChatPromptTemplate.from_messages([hashtag_system_template, hashtag_user_template])


# 2. Initialize the LLM (using OpenAI GPT as an example)
API_KEY = "up_NZqIrRnR6XRpI93EZKtO3UlJhtMWI"
llm = ChatUpstage(api_key=API_KEY, model_name="solar-pro", temperature=0.7)

# 3. Create the LangChain using the template and LLM
sentence_chain = LLMChain(llm=llm, prompt=sentence_prompt_template)
hashtag_chain = LLMChain(llm=llm, prompt=hashtag_prompt_template)

In [35]:
# 4. Function to process the text and get the result
def get_author_message(text_description, text_key_sentences):
    return sentence_chain.run({
            "text_description": text_description,
            "text_key_sentences": text_key_sentences
        }
    )

def get_hashtags(text_description, text_key_sentences):
    return hashtag_chain.run({
            "text_description": text_description,
            "text_key_sentences": text_key_sentences
        }
    )

# 5. Function to process multiple books
def process_multiple_books(text_description, text_key_sentences, isbn_list):
    results = {}
    for idx, (text_ds, text_ks) in enumerate(zip(text_description, text_key_sentences)):
        print(f"Processing book {idx}...")
        message = get_author_message(text_ds, text_ks)
        hashtags = get_hashtags(text_ds, text_ks)
        results[f"Book {idx}"] = {
            "message": message,
            "hashtags": hashtags,
            "isbn13": isbn_list[idx]
        }

        # 파일 저장 경로 설정
        file_path = "notebook/data"
        os.makedirs(file_path, exist_ok=True)  # 디렉토리가 없으면 생성

        # JSON 파일로 저장
        output_file = os.path.join(file_path, "llm_output.json")
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(results, f, ensure_ascii=False, indent=4)

        print(f"Results have been saved to {output_file}")

    return results

### 책 데이터로 부터 리스트 생성

In [36]:
import json
import re

# JSON 데이터 로드
with open('/Users/hyo-cheolahn/Documents/Projects/Book-Recommendation/data/scraping/all_book_data_ver3.json', "r", encoding="utf-8") as file:
    data = json.load(file)

print(len(data))
# description과 key_sentences를 통합한 리스트 생성
description_list = []
key_sentences_list = []
isbn_list = []
isbn_loss_idx = []

for idx, el in enumerate(data):
    # description과 key_sentences를 가져옴
    description = el.get('description', '')  # description이 없으면 빈 문자열
    key_sentences = el.get('key_sentences',)  # key_sentences가 없으면 빈 문자열
    isbn_str = str(el.get('isbn13', ''))  # isbn13을 문자열로 변환

    # 숫자만 추출하고 빈 문자열인 경우 처리
    isbn_numeric = re.sub(r"\D", "", isbn_str)
    if isbn_numeric:  # 숫자가 있는 경우에만 int 변환
        isbn13 = int(isbn_numeric)
    else:
        isbn13 = None
        isbn_loss_idx.append(idx)


    # 리스트에 추가
    description_list.append(description)
    key_sentences_list.append(key_sentences)
    isbn_list.append(isbn13)

# 결과 확인
print(f"총 {len(key_sentences_list)}개의 항목이 생성되었습니다.")
print(description_list[:1])
print(key_sentences_list[:1])  # 샘플 출력

16397
총 16397개의 항목이 생성되었습니다.
['행복을 찾는 방법이 아니라불행에 대한 수비력을 길러주는58가지 인생 이야기《1cm 다이빙》, 《홈 in 홈》으로 많은 사랑을 받았던 에세이스트 태수가 2년 만의 신작 《어른의 행복은 조용하다》로 돌아왔다. 이번 신작에서 저자 태수는 그동안 선보였던 이야기보다 한층 성숙하고 현명하게 삶의 행복에 가까워지는 방법에 대해 이야기한다. 새로운 것, 짜릿한 것, 남들보다 높은 곳에서 행복하고 싶어 발버둥치는 사람들에게 행복은 꼭 그런 데에만 있는 게 아니라 불행해지지 않는 것에서부터 출발한다고 조용히 일러준다.《어른의 행복은 조용하다》는 요란한 세상에서도 흔들리지 않고 내 삶을 살아가는 튼튼하고 단단한 태도를 담아냈다. 시끌벅적 기쁜 일을 찾아다니기보다도, 울 일이 없고 별다른 나쁜 일이 없는 하루를 만들어야 한다고 강조한다. 그랬을 때에야 비로소 진정한 행복이 우리 곁에 온다는 사실을 다시 한번 일깨워준다.이 책을 먼저 읽은 독자들의 반응 역시 뜨겁다. “울고 싶어지는 날이면 태수 작가의 글을 찾는다. 충분히 울고 다시 나아가기 위해”, “『불편한 편의점』 이후로 오랜만에 끝나지 않길 바라던 책”이라며 극찬하하며, 저자의 SNS에서 5만여 명의 독자에게 선보인 선공개 원고에도 빨리 책으로 출간해달라는 요청이 쇄도했다.이제 당신 차례다. 《어른의 행복은 조용하다》를 읽고 현명하게 행복을 찾아보자. 혹자는 여전히 의심스러운 목소리로 그토록 조용한 인생에서도 행복을 발견할 수 있냐고 묻지만 저자는 확신에 찬 목소리로 단호하게 답한다. “물론”이라고.']
['그냥 지금처럼 살아라. 그렇게 살되 어떤 감정조차 책임질 수 없을 만큼 힘든 날, 마음속이 온통 타인의 감정으로 가득해 당장이라도 터져버릴 것 같은 그런 날. 부러 나밖에 없는 공간으로 도망가자. 그 조용한 공간에서 자신에게도 이렇게 말할 기회를 주자.“나 안 괜찮아.” 가끔은 남에게 줬던 섬세함을 나에게도 허락하자.포기가 습관이 되면 포기하지 않아도 되는 것까지 

In [37]:
print(len(data))
print(len(description_list))
print(len(key_sentences_list))
print(len(isbn_list))
print(isbn_loss_idx)
# print(data[4884])

16397
16397
16397
16397
[4885, 5737, 5744, 7341, 7726, 8890, 8931, 10710, 10711, 10712, 10859, 13381, 13406, 13432]
{'title': '엄마가 호호호 웃으면 마음 끝이 아렸다', 'description': '“사랑을 알아들으세요?”가족 간의 평범한 일상 모두가 ‘사랑’이었음을 깨닫는 순간들일생을 살아가며 어느 정도의 시간을 채우지 않으면 절대 깨닫지 못하는 것들이 있다. 특히 가족 간의 ‘사랑’이 그렇다. 평범한 일상과 대화, 심지어 미움과 오해로 인한 다툼의 순간마저도 모두 사랑이었음을 우리는 뒤늦게 깨닫는다. 그러고는 정현종 시인의 시 제목처럼 “사랑할 시간이 많지 않다.”는 사실에 아쉬워하고 슬퍼한다.책 『엄마가 호호호 웃으면 마음 끝이 아렸다』는 평범한 일상 속에서 찾아낸 깨달음을 잔잔한 목소리로 담아낸 에세이집이다. 저자는 자식의 입장에서, 때로는 부모의 입장에서, 배우자 또는 이웃의 입장에서 삶의 모습을 그려 나간다. 막힘없이 읽히는 가운데에서도 불쑥 수면 위로 떠오르는 시적(詩的) 문장과 아포리즘(aphorism)은 작가의 역량이 얼마나 뛰어나고, 열정이 얼마나 큰지 보여준다.누군가에게 삶은 기적이고, 누군가에게 삶은 지옥이다. 그리고 그러한 상황을 만드는 것은 적지 않은 부분이 본인의 ‘의지’에 달려있다. 따뜻한 시선으로 바라보고, 아름다운 목소리로 노래한다면 힘겨운 일상은 언제든 행복한 순간으로 뒤바뀔 수 있다. 이 책이 지친 하루하루를 살아가는 이들을 도와 행복한 미래를 앞당겨 주기를 기대해 본다.', 'key_sentences': '이 세상에 홀로 발을 딛고 선 채 옆에 있는 가족이 힘들지 않을 만큼 어깨를 기대 본다. 또한 내 어깨도 느슨하게 늘어뜨려 내어 본다. 살아가는 동안 서로 기대고 챙기며 아껴야 할 것은 바로 그 마음이라 여긴다. ‘어차피 인생은 혼자잖아. 그렇지?’ 물으면서.---「마흔의 외동딸」중에서나중에 엄마는 이 일을 두고 이렇게 말

### 예외처리 및 실행

In [38]:

# Example usage
if __name__ == "__main__":

    if len(description_list) != len(data) or len(key_sentences_list) != len(data):
        print("Warning: The length of text_description and text_key_sentences is not the same.")
        min_length = min(len(description_list), len(key_sentences_list))
        print(f"Processing only the first {min_length} items.")
    else:
        min_length = len(description_list)

    ### 테스트 개수 설정
    min_length = 5

    try:
        text_description = description_list[:min_length]
        text_key_sentences = key_sentences_list[:min_length]
    except Exception as e:
        print(f"An error occurred while processing the books: {e}")

    results = process_multiple_books(text_description, text_key_sentences, isbn_list)

    
    for book, data in results.items():
        print(f"{book} 메시지: {data['message']}")
        print(f"{book} 해시태그: {data['hashtags']}")

Processing book 0...
Results have been saved to notebook/data/llm_output.json
Processing book 1...
Results have been saved to notebook/data/llm_output.json
Processing book 2...
Results have been saved to notebook/data/llm_output.json
Processing book 3...
Results have been saved to notebook/data/llm_output.json
Processing book 4...
Results have been saved to notebook/data/llm_output.json
Book 0 메시지: "오늘은 실패했지만, 내일은 웃으며 다시 시작."
Book 0 해시태그: #어른의행복 #조용한행복 #행복의본질 #삶의태도 #긍정적인삶 #어른의삶 #행복한일상 #행복의가치 #인생의진실 #행복찾기
Book 1 메시지: 감각의 시작은 마음가짐. 10억 원짜리 의뢰처럼 일하라.
Book 1 해시태그: #일의감각
#디자이너조수용
#브랜드감각
#브랜드스토리
#디자인철학
#브랜드전문가
#브랜드에세이
#일에대한고찰
#감각의핵심
#브랜드마케팅
Book 2 메시지: 마음 감기에 자주 걸리니, 사랑을 열 배로 셀게요.
Book 2 해시태그: #문상훈
#산문집
#빠더너스
#하이퍼리얼리즘
#콩트와코미디
#자기표현
#소통의어려움
#청춘
#시인
#새치기
Book 3 메시지: 언어는 공부가 아니라 덕질입니다. 즐기는 것이 우선, 나머지는 따라오겠죠?
Book 3 해시태그: #일본어덕질 #일본어학습 #일본어회화 #유튜브일본어 #일본어책 #일본어교재 #일본어권강사 #일본어덕업성 #일본어에세이 #일본어공부법
Book 4 메시지: 죽음은 삶의 일부입니다. 오늘을 충실히 살아갈 이유입니다.
Book 4 해시태그: #죽음공부 #웰빙 #웰다잉 #삶과죽음 #존엄한죽음 #죽음준비 #삶의의미 #죽음의